In [ ]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [ ]:
VERSION = 3
MODEL_BASE_NAME = "MegaClassifier_a_EfficientNetB5"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [ ]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (456, 456)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [ ]:
OPTIMIZERS = {"Adam": lambda x: tf.keras.optimizers.legacy.Adam(learning_rate=x)}

LEARNING_RATES = [3e-4, 1e-4]

In [ ]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

In [ ]:
BATCH_SIZE = 16

In [ ]:
LOSS_FUNCTIONS = {
    "BinaryFocalCrossentropy": lambda x, y: tf.keras.losses.BinaryFocalCrossentropy(
        alpha=x, gamma=y
    ),
    "SigmoidFocalCrossEntropy": lambda x, y: tfa.losses.SigmoidFocalCrossEntropy(
        alpha=x, gamma=y
    ),
}

ALPHAS = [0.25, 0.4, 0.6, 0.75]
GAMMAS = [1.0, 2.0, 3.0]

In [ ]:
SUBVERSION = 0
RATE_SUBVERSION = 0
ALPHA_SUBVERSION = 0
GAMMA_SUBVERSION = 0

for LOSS_FUNCTION in LOSS_FUNCTIONS:
    for LEARNING_RATE in LEARNING_RATES:
        for ALPHA in ALPHAS:
            for GAMMA in GAMMAS:
                LOGS_PATH = os.path.abspath(
                    f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"
                )
                MODELS_PATH = os.path.abspath(
                    f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"
                )
                REPORTS_PATH = os.path.abspath(
                    f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"
                )

                MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}"

                train_datagen = ImageDataGenerator(
                    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
                )
                train_images = train_datagen.flow_from_dataframe(
                    dataframe=train_dataset,
                    x_col="file_name",
                    y_col="binary_label",
                    target_size=IMAGE_SIZE,
                    batch_size=BATCH_SIZE,
                    class_mode="binary",
                    shuffle=True,
                    seed=SEED,
                )

                datagen = ImageDataGenerator(
                    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
                )
                validation_images = datagen.flow_from_dataframe(
                    dataframe=validationtrain_dataset,
                    x_col="file_name",
                    y_col="binary_label",
                    target_size=IMAGE_SIZE,
                    batch_size=BATCH_SIZE,
                    class_mode="binary",
                    shuffle=True,
                    seed=SEED,
                )
                test_images = datagen.flow_from_dataframe(
                    dataframe=test_dataset,
                    x_col="file_name",
                    y_col="binary_label",
                    target_size=IMAGE_SIZE,
                    batch_size=BATCH_SIZE,
                    class_mode="binary",
                    shuffle=False,
                    seed=SEED,
                )

                pretrained_model = tf.keras.applications.EfficientNetB5(
                    weights="imagenet",
                    include_top=False,
                    input_shape=IMAGE_SHAPE,
                )
                pretrained_model.trainable = False

                model = tf.keras.Sequential(
                    [
                        pretrained_model,
                        tf.keras.layers.GlobalAveragePooling2D(),
                        tf.keras.layers.Dense(1, activation="sigmoid"),
                    ],
                    name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
                )

                model.compile(
                    optimizer=OPTIMIZERS["Adam"](x=LEARNING_RATE),
                    loss=LOSS_FUNCTIONS[LOSS_FUNCTION](x=ALPHA, y=GAMMA),
                    metrics=METRICS,
                )

                history = model.fit(
                    train_images,
                    epochs=EPOCHS,
                    validation_data=validation_images,
                    callbacks=[TensorBoard(log_dir=LOGS_PATH)],
                )

                dataframe = pd.DataFrame(history.history)
                history_path = os.path.join(
                    LOGS_PATH,
                    f"history_v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.csv",
                )
                dataframe.to_csv(history_path, sep=";", index=False)

                os.makedirs(REPORTS_PATH, exist_ok=True)
                accuracy_chart = graphics.create_training_accuracy_chart(
                    history_path=history_path,
                    model_name=MODEL_COMPLETE_NAME,
                )
                accuracy_chart.write_image(
                    f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
                )

                loss_chart = graphics.create_training_loss_chart(
                    history_path=history_path,
                    model_name=MODEL_COMPLETE_NAME,
                )
                loss_chart.write_image(
                    f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
                )

                os.makedirs(MODELS_PATH, exist_ok=True)
                model.save(MODELS_PATH)

                results = model.evaluate(test_images)
                metric_names = history.model.metrics_names
                evaluation_results = {
                    ("test_" + name): value
                    for name, value in zip(metric_names, results)
                }

                evaluation = pd.DataFrame([evaluation_results])
                evaluation.to_csv(
                    os.path.join(
                        LOGS_PATH,
                        f"evaluation_v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.csv",
                    ),
                    sep=";",
                    index=False,
                )

                y_pred_prob = model.predict(test_images)
                y_true = test_images.labels

                fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
                roc_auc = auc(fpr, tpr)

                roc_curve_chart = graphics.create_roc_curve_chart(
                    fpr=fpr,
                    tpr=tpr,
                    roc_auc=roc_auc,
                    model_name=MODEL_COMPLETE_NAME,
                )
                roc_curve_chart.write_image(
                    f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
                )

                optimal_idx = np.argmax(tpr - fpr)
                optimal_threshold = thresholds[optimal_idx]

                # y_pred_prob = y_pred_prob.flatten()
                y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
                conf_matrix = confusion_matrix(y_true, y_pred_class)
                conf_matrix_text = [
                    [str(value) for value in row] for row in conf_matrix
                ]

                # Crear el gráfico correctamente
                confusion_matrix_chart = graphics.create_confusion_matrix_chart(
                    conf_matrix=conf_matrix,
                    conf_matrix_text=conf_matrix_text,
                    model_name=MODEL_COMPLETE_NAME,
                )
                confusion_matrix_chart.write_image(
                    f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
                )

                y_pred_class = (y_pred_prob > 0.5).astype(int)
                conf_matrix = confusion_matrix(y_true, y_pred_class)
                conf_matrix_text = [
                    [str(value) for value in row] for row in conf_matrix
                ]

                confusion_matrix_chart = graphics.create_confusion_matrix_chart(
                    conf_matrix=conf_matrix,
                    conf_matrix_text=conf_matrix_text,
                    model_name=MODEL_COMPLETE_NAME,
                )
                confusion_matrix_chart.write_image(
                    f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.{RATE_SUBVERSION}.{ALPHA_SUBVERSION}.{GAMMA_SUBVERSION}.png"
                )

                GAMMA_SUBVERSION += 1

            GAMMA_SUBVERSION = 0
            ALPHA_SUBVERSION += 1

        RATE_SUBVERSION += 1
        GAMMA_SUBVERSION = 0
        ALPHA_SUBVERSION = 0

    SUBVERSION += 1
    RATE_SUBVERSION = 0
    ALPHA_SUBVERSION = 0
    GAMMA_SUBVERSION = 0
    print("\n\n")
